In [ ]:
#Libraries
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import precision_score
from sklearn.ensemble import RandomForestClassifier
import sklearn.linear_model

##### yfinance is a popular open source library developed by Ran Aroussi as a means to access the financial data available on Yahoo Finance.






# Downloading the Dataset

*   Convert to csv
*  Convert Date to Datetime

*   Other Basic Cleaning




In [ ]:
import pandas as pd
import yfinance as yf

# Create a Ticker object for the S&P 500 index (^GSPC) using the Yahoo Finance API
sp500 = yf.Ticker("^GSPC")

# Retrieve historical market data for the S&P 500 index from Yahoo Finance, covering the maximum available period
# The period parameter specifies the timeframe for which historical data is requested. Here, "max" indicates
# that data for the maximum available period will be retrieved.
sp500 = sp500.history(period="max")
sp500.index = sp500.index.date

# Convert filter date to datetime object
filter_date = pd.to_datetime('2023-12-31')

# Filter data up to December 31, 2023
sp500_filtered = sp500.loc[sp500.index <= filter_date]

# Download it as a CSV file
sp500csv = sp500_filtered.to_csv("stock_market_data.csv")

# Load the stock market data from a CSV file into a pandas DataFrame
sp500_downloaded = pd.read_csv("stock_market_data.csv", index_col=0)

# Convert the index of the DataFrame to datetime format
sp500_downloaded.index = pd.to_datetime(sp500_downloaded.index)

print(sp500_downloaded)


# Setting up the target



*   Decrease the size of the DataFrame
*   Delete irrelevant fields
*   Create new fields to determine the target



In [5]:
#Looking at too much data would make the model more inaccurate.
#The market changes over time and the market shift would make old data worsen the model
#To prevent this, data from after 1990 is taken

specificDate = pd.to_datetime("1990-01-01").date()
sp500_downloaded = sp500_downloaded.loc[specificDate:].copy()

#Deleting these columns from the data frame as they are for individual stocks and not required to index
del sp500_downloaded["Dividends"]
del sp500_downloaded["Stock Splits"]

# creating a column that shows the next day's closing price
sp500_downloaded["Tomorrow"] = sp500_downloaded["Close"].shift(-1)
# creating the target column that uses the "Tomorrow" column to see if the price increases the next day
sp500_downloaded["Target"] = (sp500_downloaded["Tomorrow"] > sp500_downloaded["Close"]).astype(int)

In [6]:
# Resulting matrix will look like this
sp500_downloaded

,Open,High,Low,Close,Volume,Tomorrow,Target
1990-01-02,353.399994,359.690002,351.980011,359.690002,162070000,358.760010,0
1990-01-03,359.690002,360.589996,357.890015,358.760010,192330000,355.670013,0
1990-01-04,358.760010,358.760010,352.890015,355.670013,177000000,352.200012,0
1990-01-05,355.670013,355.670013,351.350006,352.200012,158530000,353.790009,1
1990-01-08,352.200012,354.239990,350.540009,353.790009,140110000,349.619995,0
...,...,...,...,...,...,...,...
2023-12-22,4753.919922,4772.939941,4736.770020,4754.629883,3046770000,4774.750000,1
2023-12-26,4758.859863,4784.720215,4758.450195,4774.750000,2513910000,4781.580078,1
2023-12-27,4773.450195,4785.390137,4768.899902,4781.580078,2748450000,4783.350098,1
2023-12-28,4786.439941,4793.299805,4780.979980,4783.350098,2698860000,4769.830078,0
